In [4]:
import matplotlib.pyplot as plt
import numpy as np
from qiime2 import Artifact
from os import listdir
from scipy.stats import chisquare
import pandas as pd

def get_first_digits(data):
    """return first digits from the data to test Benford's law
    data -- a list of positive integers(zeros and negative numbers will be skipped)
    
    """
    
    data = [x for x in data if pd.notna(x) and x > 0]
    first_digits = [int(str(x)[0]) for x in data]
    return first_digits


def frequency_fdigits (first_digits):
    """return frequencies of the first digits
    first_digits -- list of first digits extracted from data
    
    """
    
    frequency_results = {}
    for first_digit in first_digits:
        count_fdigit = first_digits.count(first_digit)
        length_of_list = len(first_digits)
        frequency = count_fdigit/length_of_list
        frequency_results[first_digit] = frequency
        return frequency_results

def benfords_law_predictions():
    """ expected probabilities for each digits based on Benford's law
        digits -- from 1-9
    """
    
    return {digit: np.log10(1 + 1/digit) for digit in range(1, 10)}

def benfords_law_test(data):
    """ return chi square and p value for the fit of the data to Benford's law
        data-- list of numbers to test the Benford's law
    """
   
    # Get the first digits
    first_digits = get_first_digits(data)
    
    # Calculate observed frequencies of first digits
    observed_freqs = np.bincount(first_digits, minlength=10)[1:10] # Skip index 0
    
    # Calculate expected frequencies according to Benford's Law
    total_count = len(first_digits)
    expected_freqs = benfords_law_predictions()
    expected_freqs = [expected_freqs[n]*total_count for n in range(1,10)]
    
    # Perform chi-square test
    print(observed_freqs,expected_freqs)
    chi2_stat, p_value = chisquare(observed_freqs, expected_freqs)
    
    return chi2_stat, p_value

def benfords_law_plot(observed_freqs, expected_freqs, output_filepath):
    """ return plots of observed and expected frequencies
        observed_freqs -- a list of observed frequencies (0.0 - 1.0) of digits 1-9
        expected_freqs -- a list of expected frequencies (0.0 - 1.0) of digits 1-9
        output_filepath -- the location you want to save your graph image
    """

    # Plot the observed vs expected distribution
    plt.bar(range(1, 10), observed_freq_normalized, alpha=0.6, color='blue', label='Observed')
    plt.plot(range(1, 10), [e / total_count for e in expected_freq], 'ro-', label='Expected (Benford)')
    plt.xlabel('First Digit')
    plt.ylabel('Frequency')
    plt.title('Benford\'s Law Test')
    plt.savefig(output_filepath)
    
# Load your data

def apply_benfords_law_to_table(feature_table_fp,axis="features",min_abundance = 0.1, min_prevalence=1/25.0):
    """Returns a pandas DataFrame of Chi-squared and p-values for fit of each feature to Benford's Law
    
    Parameters
    ==========
    
    feature_table_fp -- a string for the path to a QIIME2 feature table .qza file
    
    axis -- a string either 'features' or 'samples' (we think features is more valid)
    
    min_abundance -- if given, filter feature table to a minimum of this many counts
    min_prevalence -- require features (i.e. microbes) to be in at least this many samples
      to be included in the analysis.
    
    """
    
    print(f"about to load the feature table: {feature_table_fp}")
    feature_table = Artifact.load(feature_table_fp)
    print(f"Loaded feature table with id: {feature_table.uuid}")
    from qiime2.plugins.feature_table.methods import filter_features_conditionally
    filtered_feature_table_results = filter_features_conditionally(table = feature_table,\
                                                               abundance = min_abundance,
                                                               prevalence = min_prevalence)
    
    filtered_feature_table = filtered_feature_table_results.filtered_table
    df = filtered_feature_table.view(pd.DataFrame)
    
    if axis == 'samples':
        df = df.T

    # Loop through each column and apply Benford's Law test
    results = {}
    for column in df.columns:
        #print(f"Processing column: {column}")
        column_data = df[column].dropna().tolist()
        chi2_stat, p_value = benfords_law_test(column_data)
        #print("p value:", p_value)
        if p_value is not None:
            results[column] = {"chi2_stat": chi2_stat, "p_value": p_value}
    
    # Convert results dictionary to a
    # DataFrame
    summary_df = pd.DataFrame(results).T
    
    return summary_df

def get_output_filepath(input_path,output_dir,suffix):
    """Return a string for the output path by combining output_dir, base filename and a suffix

    parameters
    ==========
    
    input_path -- a string for the input filepath (e.g. "../input/feature_table.qza")
    output_dir -- a string for the output directory (e.g. "../output")
    suffix -- a string to add to the base filename part of the input path to get an output
    filename (e.g. "results.txt")
    """
    print("Generating output filepath for input path:",input_path)
    
    #Remove the folder (e.g. the ../input/ in ../input/feature_table.qza)
    folder,base_filename = split(input_feature_table_path)

    #Remove the file extension (e.g. the .qza in feature_table.qza)
    base_filename,extension = splitext(base_filename)
    print("Base filename:",base_filename)

    #Make an output filepath by combining output dir, base_filename and suffix
    output_filepath = join(output_dir,f"{base_filename}{suffix}")

    print(f"Output filepath: {output_filepath}")
    return output_filepath

In [6]:
from os.path import split,splitext,join

input_dir = join("..","input")
for compartment in ["mucus","tissue","skeleton"]:
    print(f"Fitting Benford's law to {compartment} microbiomes!")
    input_feature_table_path = join(input_dir,f"{compartment}_feature_table.qza")
    summary_df = apply_benfords_law_to_table(input_feature_table_path,axis="sample")
    output_dir = "../output"
    suffix = "_benfords_law_results.csv"
    output_filename = get_output_filepath(input_feature_table_path,output_dir,suffix)
    
    print("Chi-squared (mean):",summary_df['chi2_stat'].mean())
    print("p-value (mean):",summary_df['p_value'].mean())
    print(summary_df.describe())
    summary_df.to_csv(output_filename)

Fitting Benford's law to mucus microbiomes!
about to load the feature table: ../input/mucus_feature_table.qza
Loaded feature table with id: e1802e58-7a83-4fb3-b97d-489375adfe86
[54 25 23 27 15 14 13 13 15] [59.904969137132255, 35.04216055208057, 24.86280858505169, 19.285092588603227, 15.757067963477338, 13.32241113649203, 11.54039744855966, 10.179351967028877, 9.105740621574354]
[30 26 21 11 14  3  6  1  4] [34.91947949702182, 20.426586050459022, 14.492893446562793, 11.241561508934545, 9.185024541524479, 7.765827597151134, 6.727065849411661, 5.93369260389623, 5.307868905038316]
[86 56 27 27 23 14 19 14 10] [83.08427880325881, 48.601187499368024, 34.48309130389078, 26.747163590223572, 21.85402390914445, 18.47731393804925, 16.00577736584154, 14.118096195477236, 12.62906739474634]
[82 30 21 30 25 12 15 12  6] [70.13998898970762, 41.029263359973726, 29.110725629733885, 22.580033030877146, 18.449230329096583, 15.59860198393288, 13.512123645801008, 11.91853773023984, 10.66149530063731]
[36 2